In [1]:
import requests
import json
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/Seyong Ryoo/OneDrive - KU Leuven/1st year/2nd semester/Modern Data Analytics/Project/Data/CSPP_PEPP_corporate_bond_holdings_20210423.csv', encoding = 'latin-1')

In [3]:
df.head()

,NCB,ISIN,ISSUER,MATURITY DATE,COUPON RATE
0,BE,BE0002239086,Elia Transmission Belgium,27/05/2024,1.375
1,BE,BE0002256254,Enodia SCRL,22/07/2026,1
2,BE,BE0002276450,Elia Transmission Belgium,07/04/2027,1.375
3,BE,BE0002280494,Groupe Bruxelles Lambert SA,23/05/2024,1.375
4,BE,BE0002285543,Fluvius System Operator CVBA,23/06/2025,2


In [4]:
template = pd.DataFrame({"Name": df['ISSUER'].unique()}).to_csv(index=False).encode('utf-8')

In [5]:
base_url = 'https://api-eit.refinitiv.com/permid/match'
parameters = {}
parameters['x-openmatch-dataType'] = 'Organization'
parameters['X-AG-Access-Token'] = 'PcPRj33niVZ579C9GPTgwlTSpGihL3lA'
parameters['Content-Type'] = 'text/plain'
parameters['x-openmatchnumberOfMatchesPerRecord'] = '5'

In [6]:
response = requests.post(base_url, headers = parameters, data = template)

In [7]:
parameters2 = {}
parameters2['Accept'] = 'application/ld+json'
parameters2['x-ag-access-token'] = 'PcPRj33niVZ579C9GPTgwlTSpGihL3lA'

In [9]:
dt = {}

for data in response.json()['outputContentResponse']:
    try:
        company_id = data['Match OpenPermID']
        industry_id = requests.get(company_id, headers = parameters2).json()["hasPrimaryIndustryGroup"]
        dt[data['Input_Name']] = requests.get(industry_id, headers = parameters2).json()['rdfs:label']
    except:
        dt[data['Input_Name']] = 'NaN'

#df2 = pd.DataFrame(list(dt.values()), columns = ['INDUSTRY'], index = list(dt.keys()))

,INDUSTRY
Elia Transmission Belgium,Electric Utilities & IPPs
Enodia SCRL,Investment Banking & Investment Services
Groupe Bruxelles Lambert SA,Construction Materials
Fluvius System Operator CVBA,Multiline Utilities
BEFIMMO SA,Residential & Commercial REITs
...,...
Esselunga S.p.A.,Food & Drug Retailing
ERG S.p.A,Electric Utilities & IPPs
RAI - RADIOTELEVISIONE ITALIANA SPA,Media & Publishing
Infrastrutture Wireless Italiane S.p.A,Telecommunications Services


The companies with industry NaN.

In [27]:
missing = [' '.join(key.split(maxsplit=2)[:-1]) for key, value in dt.items() if value == 'NaN']

Entity search

In [32]:
template2 = pd.DataFrame({"Name": missing}).to_csv(index=False).encode('utf-8')

In [33]:
dt2 = {}

response2 = requests.post(base_url, headers = parameters, data = template2)
for data in response2.json()['outputContentResponse']:
    try:
        company_id = data['Match OpenPermID']
        industry_id = requests.get(company_id, headers = parameters2).json()["hasPrimaryIndustryGroup"]
        dt2[data['Match OrgName']] = requests.get(industry_id, headers = parameters2).json()['rdfs:label']
    except:
        dt2[data['Input_Name']] = 'NaN'

In [47]:
[key for key, value in dt2.items() if value == 'NaN']

['TRATON Finance',
 'SIX Finance',
 'Nederlandse Gasunie,',
 'HeidelbergCement Fin.Lux.',
 'Richemont Intl',
 'CTP',
 'Simon International',
 'LSEG Netherlands',
 'Daimler Intl',
 'BMW Finance',
 'Vier Gas',
 'Red Eléctrica',
 'Iberdrola International',
 'ACS Servicios',
 'ACS, Act.de',
 'El Corte',
 'Compagnie Fin.',
 'RTE EDF',
 'SOCIETE ANONYME',
 'LVMH Moët',
 'INFRA',
 'TOTAL INFRASTRUCTURES',
 'SOCIETE DES',
 'Holding d',
 'Ceetrus',
 'Auchan Holding',
 'Worldline S.A.,',
 'Terna Rete',
 'Snam']